<a href="https://colab.research.google.com/github/mnslarcher/cs224w-machine-learning-with-graphs/blob/main/notebooks/01_introduction_machine_learning_for_graphs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>